In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
import traceback
import time

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [3]:
labels = {
    "warrior_ii": "Warrior II Pose",
    "warrior_i": "Warrior I Pose",
    "t_pose": "T Pose",
    "tree_pose": "Tree Pose",
    "downward_dog": "Downward Dog Pose",
    "cobra_pose": "Cobra Pose",
    "mountain_pose": "Mountain Pose",
    "child_pose": "Child's Pose",
    "bridge_pose": "Bridge Pose",
    "triangle_pose": "Triangle Pose",
    "plank_pose": "Plank Pose",
    "Uttanasana": "Uttanasana",
    "Reverse Warrior": "Reverse Warrior",
    "Garland Pose": "Garland Pose",
    "Dog Split": "Downward facing dog split",
    "Staff Pose": "Staff Pose",
    "sukasana": "Sukasana",
    "Cobbler's Pose": "Cobbler's Pose",
    "Seated forward bend": "Seated forward bend",
    "Upavistha Konasana": "Upavistha Konasana",
    "Happy Baby Pose": "Happy Baby Pose"
}

In [ ]:
cap = cv2.VideoCapture(0)

try:
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark

                # Get coordinates
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                # Calculate angles between the joints
                left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
                right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
                left_hip_angle = calculate_angle(left_knee, left_hip, left_shoulder)
                right_hip_angle = calculate_angle(right_knee, right_hip, right_shoulder)
                left_knee_angle = calculate_angle(left_ankle, left_knee, left_hip)
                right_knee_angle = calculate_angle(right_ankle, right_knee, right_hip)

                # Classify poses
                label = "Unknown Pose"

                if left_elbow_angle > 165 and left_elbow_angle < 195 and right_elbow_angle > 165 and right_elbow_angle < 195:
                    if left_shoulder_angle > 80 and left_shoulder_angle < 110 and right_shoulder_angle > 80 and right_shoulder_angle < 110:
                        if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:
                            if left_knee_angle > 90 and left_knee_angle < 120 or right_knee_angle > 90 and right_knee_angle < 120:
                                label = "warrior_ii"
                                engine = pyttsx3.init()
                                engine.say("Warrior II Pose")
                                engine.runAndWait()

                        if left_knee_angle > 160 and left_knee_angle < 195 and right_knee_angle > 160 and right_knee_angle < 195:
                            label = "t_pose"
                            engine = pyttsx3.init()
                            engine.say("T Pose")
                            engine.runAndWait()

                if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:
                    if left_knee_angle > 315 and left_knee_angle < 335 or right_knee_angle > 25 and right_knee_angle < 45:
                        label = "tree_pose"
                        engine = pyttsx3.init()
                        engine.say("Tree Pose")
                        engine.runAndWait()

                if left_shoulder_angle > 160 and left_shoulder_angle < 200 and right_shoulder_angle > 160 and right_shoulder_angle < 200:
                    if left_hip_angle > 140 and left_hip_angle < 200 and right_hip_angle > 140 and right_hip_angle < 200:
                        if left_elbow_angle > 160 and left_elbow_angle < 200 or right_elbow_angle > 160 and right_elbow_angle < 200:
                            if left_knee_angle > 160 and left_knee_angle < 200 or right_knee_angle > 160 and right_knee_angle < 200:
                                label = "mountain_pose"
                                engine = pyttsx3.init()
                                engine.say("Mountain Pose")
                                engine.runAndWait()

                if left_elbow_angle > 160 and left_elbow_angle < 200 or right_elbow_angle > 160 and right_elbow_angle < 200:
                    if left_shoulder_angle > 160 and left_shoulder_angle < 200 or right_shoulder_angle > 160 and right_shoulder_angle < 200:
                        if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:
                            if left_knee_angle > 75 and left_knee_angle < 120 or right_knee_angle > 90 and right_knee_angle < 120:
                                label = "warrior_i"
                                engine = pyttsx3.init()
                                engine.say("Warrior I Pose")
                                engine.runAndWait()

                if left_shoulder_angle > 160 and left_shoulder_angle < 220 or right_shoulder_angle > 160 and right_shoulder_angle < 220:
                    if left_hip_angle > 0 and left_hip_angle < 50 or right_hip_angle > 0 and right_hip_angle < 50:
                        if left_elbow_angle > 160 and left_elbow_angle < 200 or right_elbow_angle > 160 and right_elbow_angle < 200:
                            if left_knee_angle > 0 and left_knee_angle < 40 or right_knee_angle > 0 and right_knee_angle < 40:
                                label = "child_pose"
                                engine = pyttsx3.init()
                                engine.say("Child's Pose")
                                engine.runAndWait()

                if left_shoulder_angle > 20 and left_shoulder_angle < 75 and right_shoulder_angle > 130 and right_shoulder_angle < 180:
                    if left_hip_angle > 135 and left_hip_angle < 180 and right_hip_angle > 75 and right_hip_angle < 135:
                        label = "Reverse Warrior"
                        engine = pyttsx3.init()
                        engine.say("Reverse Warrior Pose")
                        engine.runAndWait()

                if left_elbow_angle > 35 and left_elbow_angle < 90 or right_elbow_angle > 35 and right_elbow_angle < 90:
                    if left_shoulder_angle > 60 and left_shoulder_angle < 115 or right_shoulder_angle > 60 and right_shoulder_angle < 115:
                        if left_knee_angle > 0 and left_knee_angle < 45 or right_knee_angle > 0 and right_knee_angle < 45:
                            if left_hip_angle > 0 and left_hip_angle < 45 or right_hip_angle > 0 and right_hip_angle < 45:
                                label = "Garland Pose"
                                engine = pyttsx3.init()
                                engine.say("Garland Pose")
                                engine.runAndWait()

                if left_elbow_angle > 140 and left_elbow_angle < 200 and right_elbow_angle > 140 and right_elbow_angle < 200:
                    if left_shoulder_angle > 160 and left_shoulder_angle < 200 and right_shoulder_angle > 160 and right_shoulder_angle < 200:
                        # if left_knee_angle > 0 and left_knee_angle < 45 and right_knee_angle > 0 and right_knee_angle < 45:
                        if left_hip_angle > 20 and left_hip_angle < 120 and right_hip_angle > 20 and right_hip_angle < 120:
                            label = "sukasana"
                            engine = pyttsx3.init()
                            engine.say("Sukasana Pose")
                            engine.runAndWait()

                if left_elbow_angle > 140 and left_elbow_angle < 200 and right_elbow_angle > 140 and right_elbow_angle < 200:
                    # if left_shoulder_angle > 30 and left_shoulder_angle < 60 and right_shoulder_angle > 30 and right_shoulder_angle < 60:
                    if left_knee_angle > 0 and left_knee_angle < 45 and right_knee_angle > 0 and right_knee_angle < 45:
                        if left_hip_angle > 40 and left_hip_angle < 90 and right_hip_angle > 40 and right_hip_angle < 90:
                            label = "Cobbler's Pose"
                            engine = pyttsx3.init()
                            engine.say("Cobbler's Pose")
                            engine.runAndWait()

                if left_elbow_angle > 140 and left_elbow_angle < 200 or right_elbow_angle > 140 and right_elbow_angle < 200:
                    if left_shoulder_angle > 60 and left_shoulder_angle < 120 or right_shoulder_angle > 60 and right_shoulder_angle < 120:
                        if left_knee_angle > 140 and left_knee_angle < 200 or right_knee_angle > 140 and right_knee_angle < 200:
                            if left_hip_angle > 20 and left_hip_angle < 50 or right_hip_angle > 20 and right_hip_angle < 50:
                                label = "Uttanasana"
                                engine = pyttsx3.init()
                                engine.say("Uttanasana")
                                engine.runAndWait()

                if left_shoulder_angle > 0 and left_shoulder_angle < 45 and right_shoulder_angle > 0 and right_shoulder_angle < 45:
                    if left_knee_angle > 140 and left_knee_angle < 200 and right_knee_angle > 140 and right_knee_angle < 200:
                        if left_hip_angle > 30 and left_hip_angle < 90 and right_hip_angle > 30 and right_hip_angle < 90:
                            label = "Upavistha Konasana"
                            engine = pyttsx3.init()
                            engine.say("Upavistha Konasana")
                            engine.runAndWait()

                if left_elbow_angle > 140 and left_elbow_angle < 200 or right_elbow_angle > 140 and right_elbow_angle < 200:
                    if left_shoulder_angle > 30 and left_shoulder_angle < 90 or right_shoulder_angle > 30 and right_shoulder_angle < 90:
                        if left_knee_angle > 70 and left_knee_angle < 120 or right_knee_angle > 70 and right_knee_angle < 120:
                            if left_hip_angle > 0 and left_hip_angle < 45 or right_hip_angle > 0 and right_hip_angle < 45:
                                label = "Happy Baby Pose"
                                engine = pyttsx3.init()
                                engine.say("Happy Baby Pose")
                                engine.runAndWait()

                if label in labels:
                    cv2.putText(image, labels[label], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(image, "Unknown Pose", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


            except Exception as e:
                print(traceback.format_exc())

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            cv2.imshow('Yoga Pose Recognition', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()
    
except:
    cap.release()
    cv2.destroyAllWindows()